### Here is what we have done in order to obtain btc_data.csv which is used to train our model

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/bitcoin-historical-data/data.csv")

In [ ]:
df_copy = df.copy()

In [ ]:
df_copy.Date = pd.to_datetime(df_copy.Date)

In [ ]:
df_copy.rename(columns={
    "Date": "date", 
    "Open": "open", 
    "High": "high", 
    "Low": "low", 
    "Close": "close", 
    "Volume BTC": "volume"
}, inplace=True)

In [ ]:
new_df = df_copy.drop(["Timestamp", "Symbol", "Volume USD"], axis=1)

In [ ]:
try:
    new_df.set_index("date", inplace=True)
except:
    pass


In [ ]:
df_5min = new_df.resample('5T').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
})

In [ ]:
df_5min_inter = df_5min.copy()

In [ ]:
df_5min_inter[["open", "close", "high", "low"]] = df_5min_inter[["open", "close", "high", "low"]].interpolate(method='linear')

In [ ]:
import matplotlib.pyplot as plt 

fig, ax = plt.subplots()
plot_data = df_5min_inter.loc["2018-11-01":"2019-01-01"]
ax.plot(plot_data.index, plot_data["open"])
plt.xticks(rotation=45)

plt.savefig("linear_interpolation.jpg", dpi=400)
plt.show()

In [ ]:
df_5min_inter = df_5min_inter.iloc[1:]

In [ ]:
df_5min_inter.head()

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

# Создадим новый DataFrame, который будет содержать ваши исходные данные OHLSW
combined_df = df_5min_inter.copy()

# Получаем все классы из tti.indicators, которые являются индикаторами
indicator_classes = [cls for _, cls in inspect.getmembers(ti, inspect.isclass)]

# Проходим по каждому индикатору и добавляем его данные в основной DataFrame
for indicator_class in indicator_classes:
    try:
        # Инициализируем индикатор с вашим OHLSW DataFrame
        indicator = indicator_class(input_data=df_5min_inter)
        
        # Получаем рассчитанные данные индикатора
        indicator_data = indicator.getTiData()

        # Присоединяем данные индикатора к основному DataFrame
        # Примечание: добавляем как новые колонки (проверяем пересечение по индексам)
        combined_df = combined_df.join(indicator_data, how='left')
        
        #print(f"Добавлен индикатор: {indicator_class.__name__}")
    except Exception as e:
        pass
        #print(f"Ошибка при вычислении {indicator_class.__name__}: {e}")

# Выводим объединённый DataFrame
combined_df.head()

In [ ]:
combined_df['price_change'] = combined_df['close'].pct_change()
combined_df.sort_index(ascending=True)
combined_df['close_target'] = combined_df['close'].shift(-1)
combined_df = combined_df[:-1]
combined_df.head()

In [ ]:
combined_df.to_csv("submission_baseline.csv", index=True)